

Need help




In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 1. Load Iris dataset and convert to PyTorch tensors
iris = load_iris()
X = torch.tensor(iris.data, dtype=torch.float32)
y = torch.tensor(iris.target, dtype=torch.long)

dataset = TensorDataset(X, y)

# 2. Set up K-Fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
fold = 1
accuracies = []

# 3. Define a simple neural network model
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 3)  # 3 classes for Iris
        )

    def forward(self, x):
        return self.net(x)

# 4. Loop through each fold
for train_idx, val_idx in kf.split(dataset):
    print(f"\nFold {fold}")

    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=16)

    model = SimpleModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Training loop
    model.train()
    for epoch in range(50):  # few epochs for simplicity
        for xb, yb in train_loader:
            optimizer.zero_grad()
            output = model(xb)
            loss = criterion(output, yb)
            loss.backward()
            optimizer.step()

    # Validation
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            output = model(xb)
            preds = torch.argmax(output, dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)

    acc = correct / total
    accuracies.append(acc)
    print(f"Accuracy: {acc * 100:.2f}%")
    fold += 1

# 5. Final result
print(f"\nAverage Accuracy: {np.mean(accuracies) * 100:.2f}%")



Fold 1
Accuracy: 96.67%

Fold 2
Accuracy: 96.67%

Fold 3
Accuracy: 96.67%

Fold 4
Accuracy: 93.33%

Fold 5
Accuracy: 100.00%

Average Accuracy: 96.67%


In [ ]:
dataset